In [1]:
import numpy as np
import os
import pandas as pd
import regex as re
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from difflib import SequenceMatcher
from collections import Counter
from functools import partial

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

# import pysbd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
# from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers.keras_callbacks import KerasMetricCallback
import evaluate
import bert_score
from sklearn.model_selection import train_test_split
df=pd.read_csv('news_summary.csv',encoding= 'ISO-8859-1')
df1=pd.read_csv('news_summary_more.csv',encoding= 'ISO-8859-1')
train_df,test_df=train_test_split(df,test_size=0.2,random_state=42)
val_df,test_df=train_test_split(test_df,test_size=0.35,random_state=42)
train_df.to_csv(r'C:\Users\User\OneDrive\Desktop\ML projects\train.csv',index=False)
test_df.to_csv(r'C:\Users\User\OneDrive\Desktop\ML projects\test.csv',index=False)
val_df.to_csv(r'C:\Users\User\OneDrive\Desktop\ML projects\valid.csv',index=False)
df=load_dataset('csv',data_files={'train': r'C:\Users\User\OneDrive\Desktop\ML projects\train.csv', 'validation': r'C:\Users\User\OneDrive\Desktop\ML projects\valid.csv', 'test': r'C:\Users\User\OneDrive\Desktop\ML projects\test.csv'})
train_set = df['train']
validation_set = df['validation']
test_set = df['test']
df['train'][0]
from datasets import load_metric
metric=load_metric('rouge')
model_checkpoint=('facebook/bart-large-cnn')
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer("hello my name is umar/")
max_length=512
max_target_length=100
def preprocess(noori):
    model_inputs=tokenizer(noori['ctext'],max_length=max_length,truncation=True,padding=True)
    with tokenizer.as_target_tokenizer():
        labels=tokenizer(noori['text'],max_length=max_target_length,padding=True,truncation=True)

    model_inputs['labels']=labels['input_ids']
    return model_inputs



# Iterate over each dataset in the DatasetDict
for dataset_name, dataset in df.items():
    # Check for null values in each column of the dataset
    null_columns = dataset.column_names  # Get all column names
    for column in null_columns:
        null_count = sum(1 for value in dataset[column] if value is None)
        if null_count > 0:
            print(f"Dataset: {dataset_name}, Column: {column}, Null Count: {null_count}")
for dataset_name,dataset in df.items():
    dataset=dataset.filter(lambda x:all(value is not None for value in x.values()))
    df[dataset_name]=dataset
tokenized_df=df.map(preprocess,batched=True)

In [14]:
model=TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size=8
learning_rate =2e-5
weight_decay=0.01
num_of_epochs=5
data_coll=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors='np')
gen_data_coll=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors='np',pad_to_multiple_of=128)
train_data=model.prepare_tf_dataset(
    tokenized_df['train'],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_coll
)
test_data=model.prepare_tf_dataset(
    tokenized_df['validation'],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_coll
)
gen_data=model.prepare_tf_dataset(
    tokenized_df['validation'],
    batch_size=8,
    shuffle=False,
    collate_fn=gen_data_coll
)
opt=AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate=weight_decay)
model.compile(optimizer=opt)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['model.decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBartForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.shared.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:

history = model.fit(
    train_data,
    epochs=num_of_epochs,
    validation_data=test_data,
    batch_size=batch_size,
)


Epoch 1/5


RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL ResourceExhaustedError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\framework\errors_impl.py(377): __init__
  C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\eager\execute.py(52): quick_execute
  C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\eager\polymorphic_function\monomorphic_function.py(381): call
  C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\eager\polymorphic_function\monomorphic_func